In [1]:
import re,requests
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from os import listdir
from os.path import isfile, join
from tqdm import tqdm

In [2]:
words = ['мм', 'рт', 'ст', 'г', 'м', 'мл', 'кг', 'мг', 'см', 'ч', 'е', 'х', 'a', 'd', 'b', 'i', 'ii', 'iii'] 
set_stopWords = stopwords.words("russian") + words
set_stopWords = set(set_stopWords)

In [3]:
def deleteStopWords(text):
    stop = ['др. греч.', 'греч.', 'лат.', 'др.', 'и т. д', 'и т.д', 'т. е.', 'т.е.', 'т.н.', 'т. н.', 'т.п.', 'т. п.',
                'т. д', 'т.д', 'мм. рт. ст.', 'мм рт. ст.', 'мм.рт.ст', 'МЕДПОИСК.РУ', 'Контактная форма ниже', 'ºС']
    for i in stop:
        text = text.replace(i, '')
    return text

In [4]:
def clearText(text):
    text = deleteStopWords(text)
    text = text.replace('\xad', '')
    text = text.replace('\n\n', '.')
    text = text.replace('..', '.')
    text = text.replace(':', ' ')
    text = text.replace(';', '.')
    text = text.replace('?', '.')
    text = text.replace('!', '.')
    text = re.sub(r'\[[\d+]*\]', '', text)
    text = re.sub(r'\([\d+]*\)', '', text)
    text = re.sub('\d+', ' ', text)
    text = re.sub('[^\w.]', ' ', text)
    
    final_s = ''
    for sen in text.split('.'):
        sen = sen.strip()
        tmp_t = ''
        for tok in sen.split(' '):
            tok = tok.strip()
            if(len(tok) <= 2):
                continue
            if(tok.lower() not in set_stopWords):
                tmp_t += tok + ' '
        if(len(tmp_t.split()) < 2):
            continue
        final_s += tmp_t + '.'
    return final_s

In [11]:
d = {}
s = requests.Session()
not_worked = []
for i in tqdm(range(1,180)):
    url = 'http://www.krasotaimedicina.ru/diseases/?PAGEN_1='+ str(i)
    page = s.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    mu = soup.find_all("div", {"class":"element elementCosEqPr disLink"})
    
    if(len(mu) == 0):
        not_worked.append('http://www.krasotaimedicina.ru/diseases/?PAGEN_1='+ str(i))
        continue
    
    for j in mu:
        topic = j.find("a").text
        link = str(j.find('a')['href'])
        
        page = s.get('http://www.krasotaimedicina.ru'+link) 
        soup = BeautifulSoup(page.content, 'lxml')
        dt = soup.find("div", {"class":"detailTextDis"})
        pdt = soup.find("div", {"class":"previewTextDis"})
        
        if(dt == None or pdt == None):
            # not_worked.append('http://www.krasotaimedicina.ru'+link)
            continue
        
        for block in [dt, pdt]:
#             for j in block.find_all('a'):
#                 j.decompose()
            try:
                for j in block.find_all('img'):
                    j.decompose()
            except Exception:
                pass
            
            try:
                for j in block.find_all('h2'):
                    j.decompose()
            except Exception:
                pass
            
            try:
                for j in block.find_all('script'):
                    j.decompose()
            except Exception:
                pass
        
        text_dt = clearText(dt.text)
        text_pdt = clearText(pdt.text)       
        d[topic]= text_dt + ' ' + text_pdt

100%|██████████| 179/179 [29:44<00:00,  7.67s/it]


In [20]:
len(d)

3565

In [16]:
name = []
descr = []

for i in d.keys():
    name.append(i)
    descr.append(d[i])

In [17]:
res = pd.DataFrame()
res['name'] = name
res['descr'] = descr

In [18]:
res.head()

,name,descr
0,Hallux valgus,Hallux valgus вальгусная деформация первого па...
1,HELLP-синдром,Хотя последние годы HELLP синдром наблюдается ...
2,Абдоминальная мигрень,Термин абдоминальная мигрень используется невр...
3,Абиотрофия сетчатки,Абиотрофия сетчатки дегенерация сетчатки дистр...
4,Абсанс,Первые упоминания абсанс датируются годом .Тер...


In [19]:
# res.to_csv('bolezni.csv', index=False)